<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/4_KoBERT_emotion_finetune_jr_70000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [1]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000_jr.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 136 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595723 sha256=723a2db9854fed128eda6662d43558f7b1fdf9f46b5b58937f1c66134c34cb5f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 30.4 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-s1c31882
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-s1c31882
     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 15.0 MB/s 
     |████████████████████████████████| 596 kB 39.9 MB/s 
     |████████████████████████████████| 61 kB 517 kB/s 
     |████████████████████████████████| 3.3 MB 33.6 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=37154712fe93e7035b624140e58ccbc249af6819b4b8981d44ee1684e7e67aa2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ai77aozj/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/train
/content
popd -> /content


In [11]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/valid
/content
popd -> /content


In [12]:
train_json_list[0]

'DJSX20003906.json.answer'

In [13]:
!cat $TRAIN_DIR/{train_json_list[0]}

언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.	언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.	3
어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.	어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.	3
나한테 언니 언니 머리 너무 까만 색 아니야?	나한테 언니 언니 머리 너무 까만 색 아니야?	0
이렇게 하는 거야 그래가지고 내가	이렇게 하는 거야 그래가지고 내가	3
왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.	왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.	2
그래서 내가	그래서 내가	4
너무 짜증이 나는 거지 나보다 못생긴 애한테 그런 소리 들으니까 그래서 내가	너무 짜증이 나는 거지 나보다 못생긴 애한테 그런 소리 들으니까 그래서 내가	6
아차 싶어가지고 언니랑 같이 미용실을 가기 시작했어.	아차 싶어가지고 언니랑 같이 미용실을 가기 시작했어.	3
요즘 언니 머리 색깔을 보면은 무지개 후려쳐.	요즘 언니 머리 색깔을 보면은 무지개 후려쳐.	6
그리고 나도 옛날에 왕년에 염색 조금 했었는데 요즘 언니 머리 색깔이 더 눈에 띄고	글고 나도 옛날에 왕년에 염색 쫌 했었는데 요즘 언니 머리 색깔이 더 눈에 띄고	4
뭔 고딩(같어.)/(같아.)	뭔 고딩(같어.)/(같아.)	3
아니 내가 이 머리 색깔 할려고 한 게 아니야 내 머리 색깔이 원래 애쉬 애쉬 애쉬 애쉬그레이라고 있어.	아니 내가 이 머리 색깔 할려고 한 게 아니야 내 머리 색깔이 원래 애쉬 애쉬 애쉬 애쉬그레이라고 있어.	4
애쉬 그레이라고 애쉬 그레이라고 있는데	애쉬 그레이라고 애쉬 그레이라고 있는데	4
이 애쉬그레이는 탈색을 열 두 번은 넘게 해야지 색깔이 나온다는 거야.	이 애쉬그레이는 탈색을 열 두 번은 넘게 해야지 색깔이 나온다는 거야.	4
그러면은	그러면은	0
에쉬 그레이 가 아니라 에이씨 그레이네.	에쉬 그레이 가 아니라 에이씨 그레

In [14]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.,언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.,3
1,어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.,어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.,3
2,나한테 언니 언니 머리 너무 까만 색 아니야?,나한테 언니 언니 머리 너무 까만 색 아니야?,0
3,이렇게 하는 거야 그래가지고 내가,이렇게 하는 거야 그래가지고 내가,3
4,왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.,왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.,2
...,...,...,...
226,안 할 수밖에 없는 거고 그러니까,안 할 수밖에 없는 거고 그니까,3
227,본인이 우리가 메이크업 아티스트가 아닌 이상,본인이 우리가 메이크업 아티스트가 아닌 이상,0
228,모든 사람의 얼굴을 다 해보지는 않잖아 어떤 사람이 이걸 떡 척보면 이게 잘 어울리...,모든 사람의 얼굴을 다 해보지는 않잖아 어떤 사람이 이걸 떡 척보면 이게 잘 어울리...,4
229,그러니까 그냥 자기가 하는 방법대로 남한테 해주면 안 어울리지 당연히 안 어울리지.,그러니까 그냥 자기가 하는 방법대로 남한테 해주면 안 어울리지 당연히 안 어울리지.,6


In [15]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [16]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

In [17]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

In [18]:
dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

In [19]:
print(len(dataset_train))
print(len(dataset_valid))

45000
15000


#KoBERT 입력 데이터로 변환#

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [23]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 3241, 5770, 1221, 5850, 3241, 5770, 3450, 1809, 1763, 5400,
        2008, 2939, 6896, 1091, 1952, 7344,  862,  830, 6834,  517,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(23, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [25]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [27]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.8602070808410645 train acc 0.3125
epoch 1 batch id 201 loss 1.221313714981079 train acc 0.5813899253731343
epoch 1 batch id 401 loss 0.9871017336845398 train acc 0.5917238154613467
epoch 1 batch id 601 loss 0.7808993458747864 train acc 0.6037333610648918
epoch 1 train acc 0.6106178977272727


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6943484042553192


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2631278932094574 train acc 0.921875
epoch 5 batch id 201 loss 0.15020395815372467 train acc 0.9445740049751243
epoch 5 batch id 401 loss 0.10585122555494308 train acc 0.9502805486284289
epoch 5 batch id 601 loss 0.059747107326984406 train acc 0.9542169301164726
epoch 5 train acc 0.956298828125


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.6962101063829788


In [28]:
torch.save(model.state_dict(), SAVEPOINT_PATH)